<a href="https://colab.research.google.com/github/mirahesham2142022/AI-Project/blob/main/Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install kagglehub tensorflow pandas numpy scikit-learn

In [7]:
import kagglehub
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import os

# Download IMDB Dataset from Kaggle
path = kagglehub.dataset_download("harikiran7/imdb-movie-review-dataset")
print("Path to dataset files:", path)


print("Files in dataset directory:", os.listdir(path))

file_path = f"{path}/Train.csv"

if not os.path.exists(file_path):
    print("Error: File not found. Check if the dataset was downloaded correctly.")
else:

    df = pd.read_csv(file_path, encoding="utf-8")
    print("Dataset loaded ")
    print(df.head())


    df = df.rename(columns={'text': 'review'})

    df['label'] = df['label'].map({0: 0, 1: 1})


    tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
    tokenizer.fit_on_texts(df['review'])  # Now using 'review' column
    sequences = tokenizer.texts_to_sequences(df['review'])
    padded = pad_sequences(sequences, maxlen=200, padding='post')


    X_train, X_test, y_train, y_test = train_test_split(padded, df['label'], test_size=0.2, random_state=42)

    print("Data preprocessing completed!")


Path to dataset files: /root/.cache/kagglehub/datasets/harikiran7/imdb-movie-review-dataset/versions/1
Files in dataset directory: ['Test.csv', 'Train.csv', 'Valid.csv']
Dataset loaded successfully!
                                                text  label
0  I grew up (b. 1965) watching and loving the Th...      0
1  When I put this movie in my DVD player, and sa...      0
2  Why do people who do not know what a particula...      0
3  Even though I have great interest in Biblical ...      0
4  Im a die hard Dads Army fan and nothing will e...      1
Data preprocessing completed!


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 64, input_length=200),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.SimpleRNN(32),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("🚀 Training the model...")
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))
print("✅ Model training completed!")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


🚀 Training the model...
Epoch 1/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 114s 109ms/step - accuracy: 0.5008 - loss: 0.6975 - val_accuracy: 0.5050 - val_loss: 0.6931
Epoch 2/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 111s 111ms/step - accuracy: 0.5092 - loss: 0.6914 - val_accuracy: 0.5412 - val_loss: 0.6855
Epoch 3/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 133s 102ms/step - accuracy: 0.5950 - loss: 0.6450 - val_accuracy: 0.6173 - val_loss: 0.6209
Epoch 4/5


In [ ]:
def predict_sentiment(text):
    seq = tokenizer.texts_to_sequences([text])
    pad_seq = pad_sequences(seq, maxlen=200, padding='post')
    pred = model.predict(pad_seq)[0][0]
    return "Positive 😊" if pred > 0.5 else "Negative 😞"

print("Testing  Predictions:")
print(predict_sentiment("This movie was fantastic! I loved it!"))
print(predict_sentiment("Worst film ever. Waste of time."))
